# Optimization algorithms

Learning Objectives

- Remember different optimization methods such as (Stochastic) Gradient Descent, Momentum, RMSProp and Adam
- Use random minibatches to accelerate the convergence and improve the optimization
- Know the benefits of learning rate decay and apply it to your optimization

### Mini-batch gradient descent

- Vectorization allows to efficiently compute on m example
- What if m = 5,000,000?
- 5,000 batch with 1,000 each $X^{\{t\}}, Y^{\{t\}}$
- This is "one epoch" of training:

$
repeat \ \{ \\
for \ t = 1, ..., 5000: \\
\ \ \ \ forward \ prob \ on \ X^{\{t\}} \\
\ \ \ \ \ \ \ \ Z^{[1]} = W^{[1]} X^{\{1\}} + b^{[1]} \\
\ \ \ \ \ \ \ \ A^{[1]} = g^{[1]}(Z^{[1]}) \\
\ \ \ \ \ \ \ \ Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]} \\
\ \ \ \ \ \ \ \ A^{[2]} = g^{[2]}(Z^{[2]}) \\
\ \ \ \ \ \ \ \ ... \\
\ \ \ \ \ \ \ \ A^{[L]} = g^{[L]}(Z^{[L]}) \\
\ \ \ \ compute \ cost \ funciton \ \mathcal{J}^{\{t\}} = \frac{1}{1000} \sum_{i=1}^{1000} \mathcal{L}(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2 \times 1000} \sum_{l=1}^{L} \|W^{[l]}\|_F^2 \\
\ \ \ \ backprob \ to \ compute \ gradients \ w.r.t \ \mathcal{J}^{\{t\}} \ using \ X^{\{t\}}, Y^{\{t\}} to \ update \ W \ and \ b \\
\}
$


### Understand mini-batch gradient descent

- mini-batch = m: Batch gradient descent. $(X^{\{1\}}, X^{\{1\}}) = (X, Y) \rightarrow$ too long per iteration
- mini-batch = 1: Stochastic gradient decent. $(X^{\{1\}}, X^{\{1\}}) = (x^{(1)}, y^{(1)}) \rightarrow$ lose the speedup from vectorization
- in practice: mini-batch is between 1 and m (not too big or too small) $\rightarrow$ fastest learning 

**Choosing mini-batch size**
- If small training set (m < 2,000): use batch gradient descent
- Typical mini-batch size: 64, 128, 256, 512, 1024
- Make sure mini-batch fit in CPU/GPU memory

### Exponentially weighted averages

$
v_t = \beta v_{t-1} + (1 - \beta)\theta_t \\
\ \ \beta = 0.9: last \ 10 \ days: \frac{1}{1-\beta} \ days \\
\ \ \beta = 0.98: last \ 50 \ days: \frac{1}{1-\beta} \ days \\
\ \ \beta = 0.5: last \ 2 \ days: \frac{1}{1-\beta} \ days \\
$

### Understand exponentially weighted averages

$
v_t = \beta v_{t-1} + (1 - \beta)\theta_t \\
v_0 = 0 \\
v_1 = \beta v_0 + (1 - \beta)\theta_1 \\
v_2 = \beta v_1 + (1 - \beta)\theta_2 \\
... \\
$

### Bias correction in exponentially weighted averages
$
v_t = \frac{v_t}{1 - \beta^t}
$


### Gradient descent with momentum

$
Momentum: \\
v_{dW} = v_{db} = 0 \\
On \ iteration \ t: \\
\ \ \ \ Compute \ dW, \ db, \ on \ current \ mini-batch \\
\ \ \ \ v_{dW} = \beta v_{dW} + (1 - \beta)dW \\
\ \ \ \ v_{db} = \beta v_{db} + (1 - \beta)db \\
\ \ \ \ Update \ W \ and \ b \\
\ \ \ \ W = W - \alpha v_{dW} \\
\ \ \ \ b = b - \alpha v_{db} \\
Hyperparameter: \ \alpha, \beta \ \ \ \ \beta=0.9 \ \approx \ last \ 10 \ iterations \\
Sometimes \ omit \ (1 - \beta)
$

### RMSprop (Root Mean Square prop)

$
RMS: \\
On \ iteration \ t: \\
\ \ \ \ Compute \ dW, \ db, \ on \ current \ mini-batch \\
\ \ \ \ s_{dW} = \beta s_{dW} + (1 - \beta)dW^2 \leftarrow small \\
\ \ \ \ s_{db} = \beta s_{db} + (1 - \beta)db^2 \leftarrow large \\
\ \ \ \ Update \ W \ and \ b \\
\ \ \ \ W = W - \alpha \frac{dW}{\sqrt{s_{dW} + \epsilon}} \\
\ \ \ \ b = b - \alpha \frac{db}{\sqrt{s_{db} + \epsilon}} \\
Hyperparameter: \ \alpha, \beta, \epsilon = 10^{-8} \\
$

### Adam optimization algorithm

$
Adam: \\
v_{dW} = v_{db} = s_{dW} = s_{db} = 0 \\
On \ iteration \ t: \\
\ \ \ \ Compute \ dW, \ db, \ on \ current \ mini-batch \\
\ \ \ \ v_{dW} = \beta_1 v_{dW} + (1 - \beta)dW,
\ \ \ \ v_{db} = \beta_1 v_{db} + (1 - \beta)db \leftarrow \ Momentum \\
\ \ \ \ s_{dW} = \beta_2 s_{dW} + (1 - \beta)dW^2,
\ \ \ \ s_{db} = \beta_2 s_{db} + (1 - \beta)db^2 \leftarrow \ RMS\\
\ \ \ \ v^{corrention}_{dW} = \frac{v_{dW}}{1 - \beta_1^t},
\ \ \ \ v^{corrention}_{db} = \frac{v_{db}}{1 - \beta_1^t}, \leftarrow \ Bias \ correction \\
\ \ \ \ s^{corrention}_{dW} = \frac{s_{dW}}{1 - \beta_2^t},
\ \ \ \ s^{corrention}_{db} = \frac{s_{db}}{1 - \beta_2^t}, \leftarrow \ Bias \ correction \\
\ \ \ \ Update \ W \ and \ b \\
\ \ \ \ W = W - \alpha \frac{v^{corrention}_{dW}}{\sqrt{s^{corrention}_{dW} + \epsilon}} \\
\ \ \ \ b = b - \alpha \frac{v^{corrention}_{db}}{\sqrt{s^{corrention}_{db} + \epsilon}} \\
Hyperparameters: \ \alpha, \beta_1, \beta_2, \epsilon \\
$

**Hyperparameter choice**
- $\alpha$: need to be tune
- $\beta_1$: default choice is 0.9 (computing moving avegange for $dW$)
- $\beta_2$: default choice is 0.999 (computing moving avegange for $dW^2$)
- $\epsilon$: default choice is $10^{-8}$

***Adam: ADAptive Moment estimation***


### Learning rate decay

$
\alpha = \frac{1}{1 \ + \ decay \ rate \ \times \ epoch \ num} \alpha_0
$

**Other learning rate decay methods**

$
\alpha = 0.95^{epoch \ num} \times \alpha_0 \leftarrow exponentially \ decay \\
\alpha = \frac{c}{\sqrt{epoch \ num}}\alpha_0 \ or \ \frac{c}{\sqrt{t}}\alpha_0
$